In [19]:
import pandas as pd
import numpy as np

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
train_set = '/content/drive/MyDrive/ML_ECO/cup98lrn.csv' # zip
valid_set = '/content/drive/MyDrive/ML_ECO/cup98val.txt'

In [265]:
df = pd.read_csv(train_set, low_memory = False)  #compression='zip'

In [268]:
df.columns[24]

'NUMCHLD'

In [272]:
df.ADI

0        177.0
1         13.0
2        281.0
3         67.0
4        127.0
         ...  
95407      0.0
95408    201.0
95409     61.0
95410     13.0
95411    355.0
Name: ADI, Length: 95412, dtype: float64

## Preprosessing
Our preprosessing will include 4 steps:
1. We will delete the columns we will not use (please see documentation to see which columns we drop)
2. Re-code /transform complex variables to easier ones
3. Encode ordinal variables using ordinal encoder and create one-hot-encoding where ordinal encoding is not suitable
4. Delete observations with missing data or additionally drop features that contain more than 50% of missing data

After each step we will check whether our target variable (TARGET_B) still has enough positive examples (value = 1)




**STEP 1: Drop Features**

In dropping features we follow certain rules, namely:
1. We drop every feature that is a date
2. We drop features that refer to the sources of information 
3. We drop features that are already covered implicitly by the other features that we will encode 
4. We drop features that have a very complex structure and would require a lot of dummy variables (like STATE)
5. We drop features that highly depend on features that we previously decided to drop (like WEALTH2)



In [251]:
# some important stuff 

yesno_variables = df.columns[56:74]

In [252]:
drop1 = df.columns[1:15].to_list() # need to drop these
drop2 = df.columns[16]#.to_list() # CLUSTER   
drop3 = df.columns[18]#.to_list() # AGEFLAG
#drop4 = df.columns[44:51].to_list() # connection to the war and war veterans , might actually be relevant even though very specific data
drop11 = df.columns[51:53].to_list() # SOLICIT variables
drop5 = df.columns[54:56].to_list() 
drop6 = df.columns[362:408].to_list() # ADATES and RFA_XX
drop7 = df.columns[413:457].to_list() # Dates of different donations received
drop8 = df.columns[473]
drop9 = df.columns[480:482].to_list() # CLUSTER and GEOCODE
drop10 = df.columns[43] # drop the source from where the data is collected
drop12 = df.columns[74] #  LIFE STYLE DATA SOURCE
drop13 = df.columns[461] # Date associated with the smallest gift to date
drop14 = df.columns[463] # Date associated with the largest gift to date
drop15 = df.columns[465:468].to_list() # Dates of different gifts received
drop16 = df.columns[409] # MAXADATE
drop17 = df.columns[24] # Number of Children, very ambiguous since many empty cells, covered by other variables implicitly
drop = drop1+drop5+drop6+drop7+drop9+drop11+drop15
df = df.drop(drop, axis = 1)
df = df.drop(drop2,axis = 1)
df = df.drop(drop3, axis = 1)
df = df.drop(drop8, axis = 1)
df = df.drop(drop10, axis = 1)
df = df.drop(drop12, axis = 1)
df = df.drop(drop13, axis = 1)
df = df.drop(drop14, axis = 1)
df = df.drop(drop17, axis = 1)

**STEP 2 : Encode**

Now we start encoding things
1. We first rename the first column to Index
2. We then extract the first letter from Domain to then encode it to one-hot encoding
3. Afterwards we change the CHILDXX variables , which will be also encoded as one-hot 
4.
5.

In [254]:
df = df.rename(columns = {'Unnamed: 0' : 'Index'}) # rename first column to the Index

In [255]:
index = 0
for x in df.DOMAIN:
    df.DOMAIN[index] = df.DOMAIN[index][0]
    index += 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [256]:
df.HOMEOWNR = df.HOMEOWNR.replace({'H': 1, 'U': 0, ' ': np.NaN})
df['CHILD03'] = df['CHILD03'].replace({' ': 'N'}) # we will assume that no indication of 
# children means no children as a category
df['CHILD07'] = df['CHILD07'].replace({' ': 'N'})	
df['CHILD12'] = df['CHILD12'].replace({' ': 'N'})
df['CHILD18'] = df['CHILD18'].replace({' ': 'N'})
df.MAJOR = df.MAJOR.replace({'X': 1, ' ': 0})
df.PEPSTRFL = df.PEPSTRFL.replace({'X': 1, ' ':0})

for item in yesno_variables:
  df[item] = df[item].replace({' ': 0, 'N':0, 'Y': 1}) # these data valuesa re a bit ambiguous since some of them do not have real N so we assume
  # that empty cells represent negative observations



one_hot_encoding = ['DOMAIN', 'CHILD03','CHILD08','CHILD12','CHILD18', 'GENDER', 'MDMAUD_R']
categorical_encoding = ['MDMAUD_A', 'MDMAUD_F']
categories = [['L', 'C', 'M', 'T'],
              [1,2,5]]

In [274]:
df.MDMAUD_A.unique()

array(['X', 'C', 'M', 'L', 'T'], dtype=object)

In [264]:
df.GENDER.unique()

array([0, 1, ' ', 'C', 'U', 'J', 'A'], dtype=object)